In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.metrics import *
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 10.7656 - accuracy: 0.5356 - precision: 0.5066 - recall: 0.3575 - auc: 0.5351
Epoch 2/50
8/8 [==============================] - 0s 52ms/step - loss: 6.3737 - accuracy: 0.5400 - precision: 0.5116 - recall: 0.4112 - auc: 0.5622
Epoch 3/50
8/8 [==============================] - 0s 52ms/step - loss: 4.7896 - accuracy: 0.6166 - precision: 0.5867 - recall: 0.6168 - auc: 0.6652
Epoch 4/50
8/8 [==============================] - 0s 52ms/step - loss: 4.1606 - accuracy: 0.6736 - precision: 0.7110 - recall: 0.5117 - auc: 0.7424
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 3.6517 - accuracy: 0.7700 - precision: 0.7583 - recall: 0.7477 - auc: 0.8470
Epoch 6/50
8/8 [==============================] - 0s 52ms/step - loss: 3.2123 - accuracy: 0.8061 - precision: 0.7783 - recall: 0.8201 - auc: 0.8976
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 2.8640 - accuracy: 0.8894 - precision: 0

8/8 [==============================] - 0s 52ms/step - loss: 3.6117 - accuracy: 0.7547 - precision_1: 0.7040 - recall_1: 0.8224 - auc_1: 0.8238
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 3.1438 - accuracy: 0.8302 - precision_1: 0.8212 - recall_1: 0.8154 - auc_1: 0.9210
Epoch 7/50
8/8 [==============================] - 0s 51ms/step - loss: 2.8253 - accuracy: 0.8992 - precision_1: 0.8733 - recall_1: 0.9182 - auc_1: 0.9512
Epoch 8/50
8/8 [==============================] - 0s 51ms/step - loss: 2.6235 - accuracy: 0.8981 - precision_1: 0.8649 - recall_1: 0.9276 - auc_1: 0.9567
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 2.4247 - accuracy: 0.9266 - precision_1: 0.9188 - recall_1: 0.9252 - auc_1: 0.9754
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 2.2714 - accuracy: 0.9419 - precision_1: 0.9310 - recall_1: 0.9463 - auc_1: 0.9745
Epoch 11/50
8/8 [==============================] - 0s 50ms/step - loss: 2.0884 - accur

Epoch 8/50
8/8 [==============================] - 0s 52ms/step - loss: 2.7111 - accuracy: 0.8916 - precision_2: 0.8538 - recall_2: 0.9276 - auc_2: 0.9579
Epoch 9/50
8/8 [==============================] - 0s 52ms/step - loss: 2.5133 - accuracy: 0.9168 - precision_2: 0.9112 - recall_2: 0.9112 - auc_2: 0.9685
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 2.4168 - accuracy: 0.8938 - precision_2: 0.8559 - recall_2: 0.9299 - auc_2: 0.9570
Epoch 11/50
8/8 [==============================] - 0s 51ms/step - loss: 2.2472 - accuracy: 0.9387 - precision_2: 0.9326 - recall_2: 0.9369 - auc_2: 0.9748
Epoch 12/50
8/8 [==============================] - 0s 50ms/step - loss: 2.0820 - accuracy: 0.9639 - precision_2: 0.9625 - recall_2: 0.9603 - auc_2: 0.9845
Epoch 13/50
8/8 [==============================] - 0s 49ms/step - loss: 1.9488 - accuracy: 0.9628 - precision_2: 0.9624 - recall_2: 0.9579 - auc_2: 0.9864
Epoch 14/50
8/8 [==============================] - 0s 52ms/step - loss: 

8/8 [==============================] - 0s 52ms/step - loss: 2.2574 - accuracy: 0.9376 - precision_3: 0.9365 - recall_3: 0.9299 - auc_3: 0.9766
Epoch 11/50
8/8 [==============================] - 0s 51ms/step - loss: 2.1571 - accuracy: 0.9441 - precision_3: 0.9456 - recall_3: 0.9346 - auc_3: 0.9730
Epoch 12/50
8/8 [==============================] - 0s 53ms/step - loss: 2.0067 - accuracy: 0.9573 - precision_3: 0.9576 - recall_3: 0.9509 - auc_3: 0.9878
Epoch 13/50
8/8 [==============================] - 0s 52ms/step - loss: 1.9383 - accuracy: 0.9430 - precision_3: 0.9372 - recall_3: 0.9416 - auc_3: 0.9808
Epoch 14/50
8/8 [==============================] - 0s 53ms/step - loss: 1.8820 - accuracy: 0.9430 - precision_3: 0.9541 - recall_3: 0.9229 - auc_3: 0.9800
Epoch 15/50
8/8 [==============================] - 0s 52ms/step - loss: 1.7432 - accuracy: 0.9704 - precision_3: 0.9652 - recall_3: 0.9720 - auc_3: 0.9889
Epoch 16/50
8/8 [==============================] - 0s 50ms/step - loss: 1.6390 - a

Epoch 13/50
8/8 [==============================] - 0s 52ms/step - loss: 2.0677 - accuracy: 0.9266 - precision_4: 0.9457 - recall_4: 0.8949 - auc_4: 0.9723
Epoch 14/50
8/8 [==============================] - 0s 51ms/step - loss: 1.9213 - accuracy: 0.9485 - precision_4: 0.9504 - recall_4: 0.9393 - auc_4: 0.9823
Epoch 15/50
8/8 [==============================] - 0s 52ms/step - loss: 1.7871 - accuracy: 0.9573 - precision_4: 0.9534 - recall_4: 0.9556 - auc_4: 0.9844
Epoch 16/50
8/8 [==============================] - 0s 52ms/step - loss: 1.7007 - accuracy: 0.9595 - precision_4: 0.9536 - recall_4: 0.9603 - auc_4: 0.9840
Epoch 17/50
8/8 [==============================] - 0s 52ms/step - loss: 1.6304 - accuracy: 0.9595 - precision_4: 0.9494 - recall_4: 0.9650 - auc_4: 0.9883
Epoch 18/50
8/8 [==============================] - 0s 52ms/step - loss: 1.5936 - accuracy: 0.9551 - precision_4: 0.9448 - recall_4: 0.9603 - auc_4: 0.9817
Epoch 19/50
8/8 [==============================] - 0s 51ms/step - loss

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),input_shape=(None,n_length,n_features))))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 54ms/step - loss: 11.0949 - accuracy: 0.5356 - precision_5: 0.5053 - recall_5: 0.4463 - auc_5: 0.5638
Epoch 2/50
8/8 [==============================] - 0s 53ms/step - loss: 6.8574 - accuracy: 0.6134 - precision_5: 0.5931 - recall_5: 0.5584 - auc_5: 0.6723
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 5.0135 - accuracy: 0.7174 - precision_5: 0.6714 - recall_5: 0.7780 - auc_5: 0.8078
Epoch 4/50
8/8 [==============================] - 0s 52ms/step - loss: 4.4151 - accuracy: 0.6386 - precision_5: 0.6244 - recall_5: 0.5748 - auc_5: 0.7138
Epoch 5/50
8/8 [==============================] - 0s 53ms/step - loss: 3.8288 - accuracy: 0.8105 - precision_5: 0.8102 - recall_5: 0.7780 - auc_5: 0.8983
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 3.3753 - accuracy: 0.8905 - precision_5: 0.8694 - recall_5: 0.9019 - auc_5: 0.9540
Epoch 7/50
8/8 [==============================] - 0s 53ms/step - loss: 3.02

Epoch 1/50
8/8 [==============================] - 0s 54ms/step - loss: 10.8975 - accuracy: 0.5465 - precision_6: 0.5217 - recall_6: 0.3925 - auc_6: 0.5905
Epoch 2/50
8/8 [==============================] - 0s 52ms/step - loss: 6.5903 - accuracy: 0.6101 - precision_6: 0.5679 - recall_6: 0.7033 - auc_6: 0.6565
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 4.9083 - accuracy: 0.7087 - precision_6: 0.6695 - recall_6: 0.7477 - auc_6: 0.7907
Epoch 4/50
8/8 [==============================] - 0s 52ms/step - loss: 4.2021 - accuracy: 0.8050 - precision_6: 0.7717 - recall_6: 0.8294 - auc_6: 0.8852
Epoch 5/50
8/8 [==============================] - 0s 52ms/step - loss: 3.7940 - accuracy: 0.8116 - precision_6: 0.7581 - recall_6: 0.8785 - auc_6: 0.9032
Epoch 6/50
8/8 [==============================] - 0s 52ms/step - loss: 3.3463 - accuracy: 0.9113 - precision_6: 0.8881 - recall_6: 0.9276 - auc_6: 0.9752
Epoch 7/50
8/8 [==============================] - 0s 53ms/step - loss: 3.04

Epoch 1/50
8/8 [==============================] - 0s 55ms/step - loss: 11.2880 - accuracy: 0.5181 - precision_7: 0.4860 - recall_7: 0.4860 - auc_7: 0.5211
Epoch 2/50
8/8 [==============================] - 0s 52ms/step - loss: 7.1232 - accuracy: 0.5367 - precision_7: 0.5069 - recall_7: 0.4276 - auc_7: 0.5474
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 5.2986 - accuracy: 0.5246 - precision_7: 0.4935 - recall_7: 0.5327 - auc_7: 0.5539
Epoch 4/50
8/8 [==============================] - 0s 52ms/step - loss: 4.4733 - accuracy: 0.5936 - precision_7: 0.6234 - recall_7: 0.3364 - auc_7: 0.6499
Epoch 5/50
8/8 [==============================] - 0s 53ms/step - loss: 3.9078 - accuracy: 0.6495 - precision_7: 0.6159 - recall_7: 0.6706 - auc_7: 0.7082
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 3.4337 - accuracy: 0.7514 - precision_7: 0.7072 - recall_7: 0.8014 - auc_7: 0.8307
Epoch 7/50
8/8 [==============================] - 0s 53ms/step - loss: 3.05

Epoch 1/50
8/8 [==============================] - 0s 54ms/step - loss: 11.2092 - accuracy: 0.4940 - precision_8: 0.4325 - recall_8: 0.2547 - auc_8: 0.5042
Epoch 2/50
8/8 [==============================] - 0s 51ms/step - loss: 6.9809 - accuracy: 0.5444 - precision_8: 0.5117 - recall_8: 0.6145 - auc_8: 0.5705
Epoch 3/50
8/8 [==============================] - 0s 52ms/step - loss: 5.1615 - accuracy: 0.6002 - precision_8: 0.5636 - recall_8: 0.6519 - auc_8: 0.6459
Epoch 4/50
8/8 [==============================] - 0s 53ms/step - loss: 4.3753 - accuracy: 0.6550 - precision_8: 0.6200 - recall_8: 0.6822 - auc_8: 0.7191
Epoch 5/50
8/8 [==============================] - 0s 53ms/step - loss: 3.7921 - accuracy: 0.7722 - precision_8: 0.7489 - recall_8: 0.7734 - auc_8: 0.8570
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 3.2989 - accuracy: 0.8784 - precision_8: 0.8453 - recall_8: 0.9065 - auc_8: 0.9409
Epoch 7/50
8/8 [==============================] - 0s 53ms/step - loss: 2.94

Epoch 1/50
8/8 [==============================] - 0s 55ms/step - loss: 11.0566 - accuracy: 0.5553 - precision_9: 0.5362 - recall_9: 0.3808 - auc_9: 0.5670
Epoch 2/50
8/8 [==============================] - 0s 53ms/step - loss: 6.7716 - accuracy: 0.6846 - precision_9: 0.6246 - recall_9: 0.8201 - auc_9: 0.7554
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 4.9657 - accuracy: 0.7831 - precision_9: 0.7995 - recall_9: 0.7173 - auc_9: 0.8766
Epoch 4/50
8/8 [==============================] - 0s 53ms/step - loss: 4.3003 - accuracy: 0.7558 - precision_9: 0.7412 - recall_9: 0.7360 - auc_9: 0.8417
Epoch 5/50
8/8 [==============================] - 0s 52ms/step - loss: 3.7467 - accuracy: 0.8543 - precision_9: 0.8186 - recall_9: 0.8855 - auc_9: 0.9344
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 3.3053 - accuracy: 0.9277 - precision_9: 0.9132 - recall_9: 0.9346 - auc_9: 0.9688
Epoch 7/50
8/8 [==============================] - 0s 53ms/step - loss: 3.02

 Accuracy: 97.729% (+/-1.392) 
 Precision: 96.355% (+/-3.555) 
 Recall: 97.955% (+/-1.325) 
 AUC: 99.710% (+/-0.159) 


In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'selu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' ,metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 0s 53ms/step - loss: 12.8103 - accuracy: 0.5323 - precision_10: 0.5011 - recall_10: 0.5491 - auc_10: 0.5482
Epoch 2/50
8/8 [==============================] - 0s 51ms/step - loss: 10.3054 - accuracy: 0.5838 - precision_10: 0.5609 - recall_10: 0.5164 - auc_10: 0.6165
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 8.1800 - accuracy: 0.6451 - precision_10: 0.6126 - recall_10: 0.6612 - auc_10: 0.6886
Epoch 4/50
8/8 [==============================] - 0s 53ms/step - loss: 6.6570 - accuracy: 0.7163 - precision_10: 0.6952 - recall_10: 0.7033 - auc_10: 0.7984
Epoch 5/50
8/8 [==============================] - 0s 53ms/step - loss: 5.6171 - accuracy: 0.8061 - precision_10: 0.7872 - recall_10: 0.8037 - auc_10: 0.8832
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 4.9009 - accuracy: 0.8729 - precision_10: 0.8750 - recall_10: 0.8505 - auc_10: 0.9430
Epoch 7/50
8/8 [==============================] - 0s 52m

Epoch 1/50
8/8 [==============================] - 0s 55ms/step - loss: 12.6373 - accuracy: 0.5597 - precision_11: 0.5312 - recall_11: 0.5164 - auc_11: 0.5681
Epoch 2/50
8/8 [==============================] - 0s 52ms/step - loss: 10.0202 - accuracy: 0.5915 - precision_11: 0.5676 - recall_11: 0.5397 - auc_11: 0.6462
Epoch 3/50
8/8 [==============================] - 0s 52ms/step - loss: 7.8812 - accuracy: 0.6889 - precision_11: 0.6636 - recall_11: 0.6822 - auc_11: 0.7460
Epoch 4/50
8/8 [==============================] - 0s 53ms/step - loss: 6.3917 - accuracy: 0.7963 - precision_11: 0.7881 - recall_11: 0.7734 - auc_11: 0.8803
Epoch 5/50
8/8 [==============================] - 0s 53ms/step - loss: 5.4030 - accuracy: 0.8729 - precision_11: 0.8498 - recall_11: 0.8855 - auc_11: 0.9473
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 4.7188 - accuracy: 0.9288 - precision_11: 0.9311 - recall_11: 0.9159 - auc_11: 0.9785
Epoch 7/50
8/8 [==============================] - 0s 53m

Epoch 1/50
8/8 [==============================] - 0s 54ms/step - loss: 12.5369 - accuracy: 0.5531 - precision_12: 0.5246 - recall_12: 0.4977 - auc_12: 0.5773
Epoch 2/50
8/8 [==============================] - 0s 52ms/step - loss: 9.9970 - accuracy: 0.6276 - precision_12: 0.5859 - recall_12: 0.7009 - auc_12: 0.6851
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 7.7889 - accuracy: 0.7273 - precision_12: 0.7126 - recall_12: 0.7009 - auc_12: 0.8156
Epoch 4/50
8/8 [==============================] - 0s 53ms/step - loss: 6.3258 - accuracy: 0.8204 - precision_12: 0.7986 - recall_12: 0.8248 - auc_12: 0.8961
Epoch 5/50
8/8 [==============================] - 0s 53ms/step - loss: 5.3555 - accuracy: 0.8686 - precision_12: 0.8532 - recall_12: 0.8692 - auc_12: 0.9444
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 4.6994 - accuracy: 0.9036 - precision_12: 0.8846 - recall_12: 0.9136 - auc_12: 0.9696
Epoch 7/50
8/8 [==============================] - 0s 53ms

Epoch 1/50
8/8 [==============================] - 0s 54ms/step - loss: 12.7269 - accuracy: 0.5455 - precision_13: 0.5200 - recall_13: 0.3949 - auc_13: 0.5620
Epoch 2/50
8/8 [==============================] - 0s 53ms/step - loss: 10.2970 - accuracy: 0.5071 - precision_13: 0.4732 - recall_13: 0.4533 - auc_13: 0.5056
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 8.1268 - accuracy: 0.5531 - precision_13: 0.5239 - recall_13: 0.5117 - auc_13: 0.5687
Epoch 4/50
8/8 [==============================] - 0s 53ms/step - loss: 6.5873 - accuracy: 0.6254 - precision_13: 0.6120 - recall_13: 0.5491 - auc_13: 0.6555
Epoch 5/50
8/8 [==============================] - 0s 53ms/step - loss: 5.5477 - accuracy: 0.6867 - precision_13: 0.6636 - recall_13: 0.6729 - auc_13: 0.7450
Epoch 6/50
8/8 [==============================] - 0s 54ms/step - loss: 4.8456 - accuracy: 0.7371 - precision_13: 0.7089 - recall_13: 0.7453 - auc_13: 0.8247
Epoch 7/50
8/8 [==============================] - 0s 52m

Epoch 1/50
8/8 [==============================] - 0s 55ms/step - loss: 12.7734 - accuracy: 0.5203 - precision_14: 0.4896 - recall_14: 0.5491 - auc_14: 0.5206
Epoch 2/50
8/8 [==============================] - 0s 53ms/step - loss: 10.2384 - accuracy: 0.5750 - precision_14: 0.5505 - recall_14: 0.5093 - auc_14: 0.5941
Epoch 3/50
8/8 [==============================] - 0s 53ms/step - loss: 8.0909 - accuracy: 0.6166 - precision_14: 0.5899 - recall_14: 0.5981 - auc_14: 0.6599
Epoch 4/50
8/8 [==============================] - 0s 52ms/step - loss: 6.5791 - accuracy: 0.7174 - precision_14: 0.6977 - recall_14: 0.7009 - auc_14: 0.7751
Epoch 5/50
8/8 [==============================] - 0s 52ms/step - loss: 5.5065 - accuracy: 0.8182 - precision_14: 0.7898 - recall_14: 0.8341 - auc_14: 0.8968
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 4.7970 - accuracy: 0.8828 - precision_14: 0.8812 - recall_14: 0.8668 - auc_14: 0.9510
Epoch 7/50
8/8 [==============================] - 0s 53m

 Accuracy: 57.642% (+/-1.680) 
 Precision: 47.588% (+/-1.000) 
 Recall: 100.000% (+/-0.000) 
 AUC: 87.086% (+/-3.846) 
